# Coregistration

- Three different images need to be coregistered: EM, 2p structural stack, and 2p video.
- 2p structural image is used as reference. EM and 2p video are coregistered to 2p structural image

In [9]:
import numpy as np
import pandas as pd

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [12]:
auth.authenticate_user()

In [17]:
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [1]:
1/0.4

2.5

## EM to 2p structural stack
- Affine transformation computed from manually annotated correspondence points.
- From $\mu m$ to $\mu m$.

In [29]:
downloaded = drive.CreateFile({'id': '1E-8BIJ-5gSkGPMs1D_7VUz-05BHB8Tkd'})
downloaded.GetContentFile('correspondence_points.csv')
correspondence_df = pd.read_csv('correspondence_points.csv')
correspondence_df

,em_x_nm,em_y_nm,em_z_nm,2p_x_um,2p_y_um,2p_z_um
0,343611.4492,250417.9579,14958.552610,185.15625,228.90625,182.0
1,355013.9059,185993.0628,2590.900730,194.53125,225.78125,113.0
2,286345.4157,182402.2280,3763.086764,139.06250,229.68750,107.0
3,289383.5417,193791.0977,3976.920274,140.62500,231.25000,118.0
4,442196.8375,225395.3290,3789.988336,266.40625,221.87500,155.0
...,...,...,...,...,...,...
507,304441.2743,183007.6088,85893.699400,146.09375,153.90625,126.0
508,240674.3707,204383.2716,84512.516840,94.53125,164.84375,147.0
509,238628.4303,275948.9703,86287.090170,89.06250,175.78125,221.0
510,264332.4059,258343.3673,86435.445200,110.15625,166.40625,200.0


In [31]:
n = correspondence_df.shape[0]

em_coords = np.zeros((n,3))
twop_coords = np.zeros((n,3))
i = 0
for i in range(n):
    
    em_coords[i,:] = np.array([correspondence_df["em_x_nm"][i],
                               correspondence_df["em_y_nm"][i],
                               correspondence_df["em_z_nm"][i]])/1000 # Divide by 1000 to convert unit to um.
    twop_coords[i,:] = np.array([correspondence_df["2p_x_um"][i],
                                 correspondence_df["2p_y_um"][i],
                                 correspondence_df["2p_z_um"][i]])

### Compute affine transformation
- This affine transformation transforms coordinates in EM space to 2p structural space. Both in physical dimension with unit $\mu m$.

In [36]:
X = np.concatenate((em_coords[:,:],np.ones((n,1))),axis=1)

trans = np.dot(np.linalg.inv(np.dot(X.T,X)), np.dot(X.T,twop_coords[:,:]))

A = trans[:3,:]
b = trans[3,:]

print(A, b)

[[ 0.81958006 -0.08841949  0.01176497]
 [-0.02841662  0.16955195  1.01670694]
 [-0.10903189 -0.89056016  0.2186504 ]] [-89.93552264 227.50461563 -79.87499339]


In [38]:
def em2struct(p):

  return np.dot(p, A) + b

### Example transformation

In [39]:
em_sample = np.array([[343611, 250417, 14959],
                      [355014, 185993, 2591],
                      [286345, 182402, 3763]])
em_sample = em_sample/1000 # Convert from nm to um.

twop_sample = em2struct(em_sample)
for i in range(em_sample.shape[0]):
  print("{} um in EM --> {} um in 2P".format(em_sample[i,:], twop_sample[i,:]))

[343.611 250.417  14.959] um in EM --> [182.93418791 226.25950707 182.03907437] um in 2P
[355.014 185.993   2.591] um in EM --> [195.45907833 225.34249304 113.96863426] um in 2P
[286.345 182.402   3.763] um in EM --> [139.15359385 229.76157338 109.76600892] um in 2P


## 2P video to 2P structural stack
- Translation that gives maximum correlation.
- From coordinate in video (256 x 256) to coordinate in structural stack (512 x 512)

### Transformation
- 9 (scans) x 3 (slices) array for each translation.
- vshift_i : video shift in x dimension.
- vshift_j : video shift in y dimension.
- sshift_i : structure stack shift in x dimension.
- sshift_j : sturcture stack shift in y dimension.

In [37]:
vshift_i = np.array([[0,0,-1],
                   [-6,-6,-6],
                   [-5,-5,-6],
                   [-6,-6,-6],
                   [-5,-5,-5],
                   [-3,-4,-4],
                   [-3,-3,-3],
                   [-3,-3,-4]])

vshift_j = np.array([[2,2,2],
                   [0,0,-1],
                   [0,-1,0],
                   [-1,-1,-1],
                   [-2,-2,-2],
                   [-3,-3,-3],
                   [-1,-1,-1],
                   [-2,-2,-2]])

sshift_i = np.array([[1,1,0],
                    [0,0,0],
                    [1,1,0],
                    [0,0,0],
                    [1,1,0],
                    [1,0,0],
                    [0,0,0],
                    [0,0,0]])

sshift_j = np.array([[1,1,1],
                    [1,1,0],
                    [1,0,1],
                    [0,0,0],
                    [0,1,0],
                    [-1,0,0],
                    [0,0,0],
                    [0,0,-1]])

In [41]:
def vid2struct(p, scan_id, slice_idx):

  scan_id -= 1
  slice_idx -= 1
  
  dxv = vshift_i[scan_id, slice_idx]
  dxs = sshift_i[scan_id, slice_idx]
  dyv = vshift_j[scan_id, slice_idx]
  dys = sshift_j[scan_id, slice_idx]

  x = (p[:,0] - dxv)*2 + dxs
  y = (p[:,1] - dyv)*2 + dys

  return np.concatenate((x.reshape(-1,1), y.reshape(-1,1)), axis=1)

### Example transformation

In [43]:
scan_id = 2 # 1~9 : 9 scans
slice_idx = 1 # 1~3 : 3 slices

vid_sample = np.array([[100,50],
                       [150,50],
                       [150,100]])

twop_sample = vid2struct(vid_sample, scan_id, slice_idx)
for i in range(vid_sample.shape[0]):
  print("{} in video --> {} in 2P".format(vid_sample[i,:], twop_sample[i,:]))

[100  50] in video --> [212 101] in 2P
[150  50] in video --> [312 101] in 2P
[150 100] in video --> [312 201] in 2P
